In [13]:
#!/usr/bin/env python

In [14]:
import wuml 

In [15]:
data = wuml.wData(xpath='../../data/wine.csv', ypath='../../data/wine_label.csv', label_type='discrete', first_row_is_label=False)

In [16]:
cf = wuml.classification(data, classifier='NaiveBayes')
print('Running a single classifier')
print(cf)

Running a single classifier
  classifier   Train
  NaiveBayes  0.9888


In [17]:
print('\n\nRun all regressors sorted by Accuracy')
results = wuml.run_every_classifier(data, y=data.Y, order_by='Test')
Y = results['GP'](data)
print(Y)



Run all regressors sorted by Accuracy


The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.


     0
0    0
1    0
2    0
3    0
4    0
..  ..
173  2
174  2
175  2
176  2
177  2

[178 rows x 1 columns]


pandas.core.frame.DataFrame

In [18]:
results['Train/Test Summary'].style

,classifier,Train,Test
,SVM,1.0,1.0
,RandomForest,1.0,1.0
,LDA,1.0,1.0
,GP,0.9938,1.0
,NaiveBayes,0.9812,1.0
,IKDR,0.9812,1.0
,NeuralNet,0.95,0.9444
,KNN,0.8312,0.8333
